In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [5]:
# Subclassing-API model
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")    
    
model5 = WideAndDeepModel()

model5.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
main_output, aux_output = model5(inputs=[input_A, input_B])
history = model5.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                     validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model5.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model5.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 52us/sample - loss: 2.1375 - output_1_loss: 1.8033 - output_2_loss: 5.1436 - val_loss: 1.1877 - val_output_1_loss: 0.8009 - val_output_2_loss: 4.6654
Epoch 2/20
11610/11610 [==============================] - 0s 37us/sample - loss: 0.9789 - output_1_loss: 0.7418 - output_2_loss: 3.1132 - val_loss: 0.8693 - val_output_1_loss: 0.6375 - val_output_2_loss: 2.9530
Epoch 3/20
11610/11610 [==============================] - 0s 35us/sample - loss: 0.8056 - output_1_loss: 0.6526 - output_2_loss: 2.1812 - val_loss: 0.7456 - val_output_1_loss: 0.5854 - val_output_2_loss: 2.1877
Epoch 4/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.7277 - output_1_loss: 0.6132 - output_2_loss: 1.7570 - val_loss: 0.6810 - val_output_1_loss: 0.5547 - val_output_2_loss: 1.8165
Epoch 5/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.6833 - output_1_loss: 0.